# 데이터 불러오기

In [ ]:
# 캐글 설치
!pip install -q kaggle

In [ ]:
# .kaggle 폴더 생성
!mkdir -p ~/.kaggle/
# kaggle.josn .kaggle로 복사
!cp kaggle.json ~/.kaggle/
# 복사 확인
!ls ~/.kaggle

In [ ]:
cd ~

In [ ]:
cd /content

In [ ]:
# 파일 권한 변경 : 읽기, 쓰기, 실행 (rwx)
!chmod 600 .kaggle/kaggle.json

In [ ]:
#downloading dataset
!kaggle datasets download -d vodan37/yolo-helmethead

In [ ]:
!unzip -q yolo-helmethead.zip

# 모델 불러오기

In [ ]:
!git clone https://github.com/ultralytics/yolov3  # 깃허브 모델을 불러온다 
%cd yolov3
!pip install -r requirements.txt  # 모델에 필요한 조건들을 설치 

# 데이터를 모델 양식에 맞춰주기

In [ ]:
# 학습에 넣을 데이터 양식
# 데이터에 맞게 모델에 넣을 양식을 작성해준다 
# 실습에 사용한 Yolo v4는 다운받은 이미지와 라벨들을 모델안에 옮겨줘야 했으나
# 
# path: ../helm/helm
# train: images/train
# val: images/valid 
# test: images/test
# #classes 
# nc: 2 
# names: ['head', 'helmet']

# coco128.yaml을 보고 양식을 직접 작성해도 되지만 코드 한번 돌려서 넣어주는게 더 편하므로 이렇게 코드를 짜보았다 
t = open('/content/yolov3/data/helm.yaml', 'w')
t.write('path: ../helm/helm\n')
t.write('train: images/train\n')
t.write('val: images/valid\n')
t.write('test: images/test\n')
t.write('nc: 2\n')
t.write("names: ['head', 'helmet']")
t.close()

# 모델 훈련

In [ ]:
# Yolo v3 모델로 우리 조의 헬멧데이터, 커스텀 데이터를 yolov3.pt, 이미 훈련된 가중치를 불러와서 전이학습을 시켰다 
# 코드 내에 yolo3.pt를 다운 받는 링크가 있어서 실행시키면 자동으로 웨이트를 다운받는다 
# 1epoch 당 30분씩 걸림 
!python train.py --img 640 --batch 16 --epochs 15 --data helm.yaml --weights yolov3.pt

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 3
wandb: You chose 'Don't visualize my results'
train: weights=yolov3.pt, cfg=, data=helm.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=5, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov3 ✅
YOLOv3 🚀 v9.6.0-16-gd58ba5e torch 1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0,

# Inference

In [ ]:
# Inference과정. 여기에 사용한 웨이트는 15epochs 학습한 가중치, best.pt를 사용했다. 원래는 run폴더안 train폴더에 exp1,2,3 등 순서대로 안에 저장이 되나 돌려서 가중치를 얻기엔 너무 오래 걸려서 
# 전에 커스텀데이터로 넣어서 돌리고 다운받은 웨이트를 사용하였다 
# detect.py를 작동하면 data의 images폴더안에 있는 이미지들을 detect를 실행하고 그 결과를 runs/detect/expn폴더에 저장한다 
!python detect.py --weights best.pt --img 640 --conf 0.25 --source data/images
!display.Image(filename='data/images/hardhat.jpg', width=600)

detect: weights=['best.pt'], source=data/images, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv3 🚀 v9.6.0-17-g3508a98 torch 1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 261 layers, 61502815 parameters, 0 gradients, 154.7 GFLOPs
image 1/4 /content/yolov3/data/images/bus.jpg: 640x480 3 heads, Done. (0.021s)
image 2/4 /content/yolov3/data/images/hardhat.jpg: 480x640 3 helmets, Done. (0.024s)
image 3/4 /content/yolov3/data/images/hardhatworker.jpg: 448x640 6 helmets, Done. (0.022s)
image 4/4 /content/yolov3/data/images/zidane.jpg: 384x640 2 heads, Done. (0.020s)
Speed: 0.3ms pre-process, 21.8ms inference, 1.2ms NMS per image at shape (1, 

In [ ]:
# 훈련시켜서 얻은 best웨이트로 valid 데이터들에 대한 검증을 실행해보았다 
# train과 detect와 마찬가지로 runs폴더에 val폴더가 생기면서 각종 평가지표와 적용 이미지들이 자동으로 저장된다  
!python val.py --weights best.pt --data helm.yaml --img 640 --iou 0.65 --half

val: data=/content/yolov3/data/helm.yaml, weights=['best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, task=val, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv3 🚀 v9.6.0-17-g3508a98 torch 1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 261 layers, 61502815 parameters, 0 gradients, 154.7 GFLOPs
val: Scanning '../helm/helm/labels/valid' images and labels...4641 found, 0 missing, 0 empty, 0 corrupted: 100% 4641/4641 [00:01<00:00, 3969.49it/s]
val: New cache created: ../helm/helm/labels/valid.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 146/146 [01:34<00:00,  1.55it/s]
                 all       4641      38272      0.949      0.932      0.966      0.608
                head       4641      25868      0.946      0.915      0.954   

In [ ]:
# Run YOLOv3 on COCO test
!python val.py --weights best.pt --data helm.yaml --img 640 --iou 0.65 --half --task test

val: data=/content/yolov3/data/helm.yaml, weights=['best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.65, task=test, device=, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv3 🚀 v9.6.0-17-g3508a98 torch 1.11.0+cu113 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 261 layers, 61502815 parameters, 0 gradients, 154.7 GFLOPs
test: Scanning '../helm/helm/labels/test' images and labels...2261 found, 0 missing, 0 empty, 0 corrupted: 100% 2261/2261 [00:00<00:00, 3937.04it/s]
test: New cache created: ../helm/helm/labels/test.cache
               Class     Images     Labels          P          R     mAP@.5 mAP@.5:.95: 100% 71/71 [00:49<00:00,  1.44it/s]
                 all       2261      19968      0.949      0.898      0.944      0.585
                head       2261      13217      0.943      0.905      0.947    